In [ ]:
#@title
!pip install -qU datasets==2.12.0 qdrant-client==1.2.0 sentence-transformers==2.2.2 torch==2.0.1
!pip install langchain sentence_transformers
!pip install rapidocr-onnxruntime
# !pip install pypdf
!pip install ctransformers
!pip install unstructured

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 7.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 619.9/619.9 MB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 46.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.8/143.8 kB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 32.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 21.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.3/849.3 kB 39.6 

In [1]:
from langchain import PromptTemplate
from langchain.embeddings import HuggingFaceEmbeddings#to get embeddings
from langchain.vectorstores import Qdrant #vector database
from qdrant_client import QdrantClient
from langchain.llms import CTransformers#to get llm
from langchain.text_splitter import RecursiveCharacterTextSplitter#splitting text into chunks
from langchain.chains import RetrievalQA#building Retrieval chain
# from langchain.document_loaders import PyPDFLoader,  UnstructuredURLLoader #to read pdfs, urls
import torch
# from datasets import load_dataset
from transformers import pipeline
from sentence_transformers import SentenceTransformer
from qdrant_client import QdrantClient
from qdrant_client.http import models
from tqdm.auto import tqdm
from typing import List
import pandas as pd

/home/akash/anaconda3/envs/QnA/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
def create_description_and_metadata(row):
    # Create description
    description = f"The product {row['product']} belongs to the category {row['category']} and subcategory {row['sub_category']}, manufactured by the brand {row['brand']}. It is priced at {row['sale_price']}, with a market price of {row['market_price']}. This product is of type {row['type']} and has a rating of {row['rating']}. Description: {row['description']}."

    # Create metadata
    metadata = {
        'name': row['product'],
        'category': row['category'],
        'sub_category': row['sub_category'],
        'brand': row['brand'],
        'sale_price': row['sale_price'],
        'market_price': row['market_price'],
        'type': row['type'],
        'rating': row['rating'],
        'decription' : row['description']
    }

    return description, metadata

# Set the file path
project_path = '/home/akash/Documents/Context-QnA/'
filepath = project_path + 'bigBasketProducts.csv'

# Read the CSV file into a DataFrame
df = pd.read_csv(filepath)

# Apply the create_description_and_metadata function to each row
df[['description', 'metadata']] = df.apply(create_description_and_metadata, axis=1, result_type='expand')

# Extract metadata list and save to a text file
metadata_list = df['metadata'].tolist()

with open(project_path + 'bigBasketProducts_metadata_list.txt', 'w') as f:
    for metadata in metadata_list:
        f.write(str(metadata) + '\n')

# Keep only the 'description' column
df = df[['description']]

# Save the updated DataFrame to a new CSV file
df.to_csv(project_path + 'dataset.csv', index=False)



In [4]:
# df.head()

,description
0,The product Garlic Oil - Vegetarian Capsule 50...
1,The product Water Bottle - Orange belongs to t...
2,"The product Brass Angle Deep - Plain, No.2 bel..."
3,The product Cereal Flip Lid Container/Storage ...
4,The product Creme Soft Soap - For Hands & Body...


In [5]:
# metadata = metadata_list
# print(metadata[0])

{'name': 'Garlic Oil - Vegetarian Capsule 500 mg', 'category': 'Beauty & Hygiene', 'sub_category': 'Hair Care', 'brand': 'Sri Sri Ayurveda ', 'sale_price': 220.0, 'market_price': 220.0, 'type': 'Hair Oil & Serum', 'rating': 4.1, 'decription': 'This Product contains Garlic Oil that is known to help proper digestion, maintain proper cholesterol levels, support cardiovascular and also build immunity.  For Beauty tips, tricks & more visit https://bigbasket.blog/'}


In [6]:
client = QdrantClient(":memory:")

In [7]:
# set device to GPU if available
device = "cuda" if torch.cuda.is_available() else "cpu"

# load the retriever model from huggingface model hub
# retriever = SentenceTransformer("multi-qa-MiniLM-L6-cos-v1", device=device)
# retriever
embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2',
                                       model_kwargs={'device': 'cpu'})

In [8]:
collections = []

In [9]:
collection_name = "BigBasket-QnA"

collections = client.get_collections()
print(collections)

# only create collection if it doesn't exist
if collection_name not in [c.name for c in collections.collections]:
    client.recreate_collection(
        collection_name=collection_name,
        vectors_config=models.VectorParams(
            size=384,
            distance=models.Distance.COSINE,
        ),
    )
collections = client.get_collections()
print(collections)

collections=[]
collections=[CollectionDescription(name='BigBasket-QnA')]


In [ ]:
# df = pd.read_csv('/content/dataset.csv')

In [ ]:
print(metadata[0])

In [ ]:
# df.head()

,description
0,The product Garlic Oil - Vegetarian Capsule 50...
1,The product Water Bottle - Orange belongs to t...
2,"The product Brass Angle Deep - Plain, No.2 bel..."
3,The product Cereal Flip Lid Container/Storage ...
4,The product Creme Soft Soap - For Hands & Body...


In [ ]:
%%time

batch_size = 256  # specify batch size according to your RAM and compute, higher batch size = more RAM usage

for index in tqdm(range(0, len(df), batch_size)):
    i_end = min(index + batch_size, len(df))  # find end of batch
    batch = df.iloc[index:i_end]  # extract batch
    embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2',
                                       model_kwargs={'device': 'cpu'})
    emb = embeddings.embed_documents(batch["description"]) # generate embeddings for batch
    meta = metadata[index:i_end]

    ids = list(range(index, i_end))  # create unique IDs

    # upsert to qdrant
    client.upsert(
        collection_name=collection_name,
        points=models.Batch(ids=ids, vectors=emb, payloads=meta),
    )

collection_vector_count = client.get_collection(collection_name=collection_name).vectors_count
print(f"Vector count in collection: {collection_vector_count}")
assert collection_vector_count == len(df)

 92%|██████████████████████████████████████▌   | 99/108 [15:13<01:32, 10.26s/it]

In [ ]:
# snapshot_info = client.create_snapshot(collection_name=collection_name)
# print(snapshot_info)

In [ ]:
# client.upload_collection(
#     collection_name=collection_name,
#     vectors=emb,
#     batch_size=64,
#     parallel=2,
# )

In [12]:
sentencetf = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

In [13]:
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

In [ ]:
from langchain.llms import GPT4All

In [15]:
def load_llm():
    llm = CTransformers(
        model = "TheBloke/Llama-2-7B-Chat-GGML",
        model_type="llama",
        temperature = 0.2
        )
    return llm

In [16]:
llm = load_llm()
llm_chain = LLMChain(llm=llm, prompt=prompt)

Fetching 1 files:   0%|                                   | 0/1 [00:00<?, ?it/s]
config.json: 100%|███████████████████████████| 29.0/29.0 [00:00<00:00, 5.95kB/s]
Fetching 1 files:   0%|                                   | 0/1 [00:00<?, ?it/s]
llama-2-7b-chat.ggmlv3.q2_K.bin:   0%|              | 0.00/2.87G [00:00<?, ?B/s]
llama-2-7b-chat.ggmlv3.q2_K.bin:   0%|     | 10.5M/2.87G [00:01<04:34, 10.4MB/s]
llama-2-7b-chat.ggmlv3.q2_K.bin:   1%|     | 21.0M/2.87G [00:01<03:38, 13.0MB/s]
llama-2-7b-chat.ggmlv3.q2_K.bin:   1%|     | 31.5M/2.87G [00:02<03:14, 14.6MB/s]
llama-2-7b-chat.ggmlv3.q2_K.bin:   1%|     | 41.9M/2.87G [00:03<03:30, 13.4MB/s]
llama-2-7b-chat.ggmlv3.q2_K.bin:   2%|     | 52.4M/2.87G [00:03<03:21, 14.0MB/s]
llama-2-7b-chat.ggmlv3.q2_K.bin:   2%|     | 62.9M/2.87G [00:04<03:09, 14.8MB/s]
llama-2-7b-chat.ggmlv3.q2_K.bin:   3%|▏    | 73.4M/2.87G [00:05<03:21, 13.9MB/s]
llama-2-7b-chat.ggmlv3.q2_K.bin:   3%|▏    | 83.9M/2.87G [00:05<03:09, 14.7MB/s]
llama-2-7b-chat.ggmlv3.q2_K.

llama-2-7b-chat.ggmlv3.q2_K.bin:  34%|██    | 975M/2.87G [01:18<02:23, 13.1MB/s]
llama-2-7b-chat.ggmlv3.q2_K.bin:  34%|██    | 986M/2.87G [01:19<02:26, 12.9MB/s]
llama-2-7b-chat.ggmlv3.q2_K.bin:  35%|██    | 996M/2.87G [01:20<02:25, 12.8MB/s]
llama-2-7b-chat.ggmlv3.q2_K.bin:  35%|█▊   | 1.01G/2.87G [01:21<02:37, 11.8MB/s]
llama-2-7b-chat.ggmlv3.q2_K.bin:  35%|█▊   | 1.02G/2.87G [01:22<02:39, 11.6MB/s]
llama-2-7b-chat.ggmlv3.q2_K.bin:  36%|█▊   | 1.03G/2.87G [01:23<02:39, 11.5MB/s]
llama-2-7b-chat.ggmlv3.q2_K.bin:  36%|█▊   | 1.04G/2.87G [01:24<02:39, 11.4MB/s]
llama-2-7b-chat.ggmlv3.q2_K.bin:  37%|█▊   | 1.05G/2.87G [01:25<02:35, 11.7MB/s]
llama-2-7b-chat.ggmlv3.q2_K.bin:  37%|█▊   | 1.06G/2.87G [01:25<02:28, 12.2MB/s]
llama-2-7b-chat.ggmlv3.q2_K.bin:  37%|█▊   | 1.07G/2.87G [01:26<02:26, 12.3MB/s]
llama-2-7b-chat.ggmlv3.q2_K.bin:  38%|█▉   | 1.08G/2.87G [01:27<02:19, 12.8MB/s]
llama-2-7b-chat.ggmlv3.q2_K.bin:  38%|█▉   | 1.09G/2.87G [01:28<02:15, 13.1MB/s]
llama-2-7b-chat.ggmlv3.q2_K.

llama-2-7b-chat.ggmlv3.q2_K.bin:  69%|███▍ | 1.99G/2.87G [02:26<00:56, 15.4MB/s]
llama-2-7b-chat.ggmlv3.q2_K.bin:  70%|███▍ | 2.00G/2.87G [02:27<00:58, 14.7MB/s]
llama-2-7b-chat.ggmlv3.q2_K.bin:  70%|███▌ | 2.01G/2.87G [02:28<00:55, 15.5MB/s]
llama-2-7b-chat.ggmlv3.q2_K.bin:  71%|███▌ | 2.02G/2.87G [02:28<00:53, 15.7MB/s]
llama-2-7b-chat.ggmlv3.q2_K.bin:  71%|███▌ | 2.03G/2.87G [02:29<00:51, 16.1MB/s]
llama-2-7b-chat.ggmlv3.q2_K.bin:  71%|███▌ | 2.04G/2.87G [02:30<00:51, 15.9MB/s]
llama-2-7b-chat.ggmlv3.q2_K.bin:  72%|███▌ | 2.06G/2.87G [02:31<00:54, 15.0MB/s]
llama-2-7b-chat.ggmlv3.q2_K.bin:  72%|███▌ | 2.07G/2.87G [02:31<00:51, 15.4MB/s]
llama-2-7b-chat.ggmlv3.q2_K.bin:  72%|███▌ | 2.08G/2.87G [02:32<00:49, 15.9MB/s]
llama-2-7b-chat.ggmlv3.q2_K.bin:  73%|███▋ | 2.09G/2.87G [02:32<00:48, 16.0MB/s]
llama-2-7b-chat.ggmlv3.q2_K.bin:  73%|███▋ | 2.10G/2.87G [02:33<00:47, 16.0MB/s]
llama-2-7b-chat.ggmlv3.q2_K.bin:  74%|███▋ | 2.11G/2.87G [02:34<00:47, 16.0MB/s]
llama-2-7b-chat.ggmlv3.q2_K.

KeyboardInterrupt: 

In [17]:
def getInfo(question: str, top_k: int) -> List[str]:
    """
    Get the relevant plot for a given question

    Args:
        question (str): What do we want to know?
        top_k (int): Top K results to return

    Returns:
        context (List[str]):
    """
    try:
        encoded_query =  sentencetf.encode(question)  # generate embeddings for the question

        result = client.search(
            collection_name=collection_name,
            query_vector=encoded_query,
            limit=top_k,
        )  # search qdrant collection for context passage with the answer

        context = [
    {
        'name': x.payload.get('name', ''),
        'category': x.payload.get('category', ''),
        'sub_category': x.payload.get('sub_category', ''),
        'brand': x.payload.get('brand', ''),
        'sale_price': x.payload.get('sale_price', 0.0),
        'market_price': x.payload.get('market_price', 0.0),
        'type': x.payload.get('type', ''),
        'rating': x.payload.get('rating', float('nan'))
    } for x in result
]
        prompt = PromptTemplate.from_template(
        "question is={question}. you should answer from: {docs}"
)

        # Chain
        llm_chain = LLMChain(llm=llm, prompt=prompt)
        answer = llm_chain(context)
        return answer

    except Exception as e:
        print({e})


llama-2-7b-chat.ggmlv3.q2_K.bin:  98%|████▉| 2.81G/2.87G [03:26<00:05, 11.0MB/s]
llama-2-7b-chat.ggmlv3.q2_K.bin:  98%|████▉| 2.82G/2.87G [03:27<00:03, 12.5MB/s]
llama-2-7b-chat.ggmlv3.q2_K.bin:  99%|████▉| 2.83G/2.87G [03:27<00:02, 13.6MB/s]
llama-2-7b-chat.ggmlv3.q2_K.bin:  99%|████▉| 2.84G/2.87G [03:28<00:01, 13.8MB/s]
llama-2-7b-chat.ggmlv3.q2_K.bin:  99%|████▉| 2.85G/2.87G [03:29<00:01, 14.2MB/s]
llama-2-7b-chat.ggmlv3.q2_K.bin: 100%|████▉| 2.86G/2.87G [03:29<00:00, 14.9MB/s]
llama-2-7b-chat.ggmlv3.q2_K.bin: 100%|█████| 2.87G/2.87G [03:30<00:00, 13.6MB/s]


In [ ]:
getInfo('what is the price of chia seeds?', 3)

In [ ]:
# custom_prompt_template = """Use the following pieces of information to answer the user’s question.
# If you don’t know the answer, just say that you don’t know, don’t try to make up an answer.

# Context: {context}
# Question: {question}

# """

# prompt = PromptTemplate(template=custom_prompt_template,
#                             input_variables=['context', 'question'])

In [ ]:
# response('')

In [ ]:
# class ConversationManager:
#     def __init__(self):
#         self.context = ""  # Initialize empty context for the conversation

#     def update_context(self, new_context):
#         self.context += " " + new_context  # Append new context to the existing context

#     def get_response(self, query):
#         self.update_context(query)  # Add the user's query to the conversation context
#         response = qa_bot_qdrant_response(query)
#         return response


In [ ]:
# conversation_manager = ConversationManager()

In [ ]:
# while True:
#     user_input = input("User: ")
#     if user_input.lower() == 'exit':
#         print("Bot: Goodbye!")
#         break
#     response = conversation_manager.get_response(user_input)
#     print("Bot:", response)